- 实现字典
- 解析式
- 迭代器

## 如何实现字典

- 拉链法
- 开地址法

实现过程：

- 传入一个key
- 就能通过这个key（索引）得到这个值，那么复杂度就是O(1)

- 初始化一个列表， 所有位置都是None
- 所有的key 都是可hash的，我们对里面的值求hash
- 假设这个列表有32个槽位，就是32位，比32位大怎么办？求模就可以了吧
- 那么这个就可以放进这个列表了

`i = hash(key) % slots # slots 槽位数`

重复的槽位 我们称   hash冲突

通过 1. 拉链法。 2.开地址法解决

### 拉链法

**字典攻击**， 黑客 通过精心设计 key 的hash值， 让所有的值 全部落在一个slot 上。 REDIS/ MEMCACHED

### 开地址法

In [1]:
d = dict([('a', 1)])

In [2]:
d

{'a': 1}

### 拉链法的实现

In [14]:
# 列表
slots = []
# 槽位数
slots_num = 32

for _ in range(slots_num):
    slots.append([])

def put(slots, key, value):
    i = hash(key) % slots_num
    
    
    for p, (k, v) in enumerate(slots[i]):
        if k == key:
            slots[i][p] = (key, value)
            break
    else:
        slots[i].append((key, value))
        return

        
    
def get(slots, key):
    i = hash(key) % slots_num
    for k, v in slots[i]:
        if k == key:
            return v
    raise KeyError(k)

In [15]:
put(slots, 'r', 2)

In [16]:
put(slots, 'd', 2)

In [17]:
slots

[[('d', 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('r', 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [7]:
get(slots, 'r')

2

In [18]:
put(slots, 'r', 3)

In [19]:
slots

[[('d', 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('r', 3)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [13]:
get(slots, 'r')

2

In [31]:
class _Dict:
    def __init__(self, num):
        self.__slots__ = []
        self.num = num
        
        for _ in range(self.num):
            self.__slots__.append([])

    def put(self, key, value):
        i = hash(key) % self.num

        for p, (k, v) in enumerate(self.__slots__[i]):
            if k == key:
                self.__slots__[i][p] = (key, value)
                break
        else:
            self.__slots__[i].append((key, value))
            return



    def get(self, key):
        i = hash(key) % self.num
        for k, v in self.__slots__[i]:
            if k == key:
                return v
        raise KeyError(k)
        
    def keys(self):
        ret = []
        for slot in self.__slots__:
            for k, _ in slot:
                ret.append(k)
        return ret

In [32]:
d = _Dict(32)

In [33]:
d.put('r', 2)

In [34]:
d.put('d', 3)

In [35]:
d.get('r')

2

In [36]:
d.get('d')

3

In [37]:
d.put('r', 4)

In [38]:
d.get('r')

4

In [39]:
d.keys()

['d', 'r']

In [40]:
d.__slots__

[[('d', 3)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('r', 4)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [42]:
hash('d') % 32

0

In [43]:
hash('r') % 32

19

In [44]:
hash(1) % 32

1

## 解析式

In [45]:
lst = list(range(10))
ret = []
for x in lst:
    ret.append(x ** 2)

In [46]:
ret

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [47]:
ret = [x ** 2 for x in lst]

In [48]:
ret

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

### 列表解析式

语法：[expr for e in iterator]

优点
- 代码简洁， 可读性高
- 效率比 普通迭代稍高，并不是数据量级的提升

In [49]:
%%timeit
lst = list(range(10000))
ret = []
for x in lst:
    ret.append(x ** 2)

3.8 ms ± 71.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%%timeit
lst = list(range(10000))
ret = [x ** 2 for x in lst]

3.08 ms ± 87.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
ret = []
for x in lst:
    if x % 2 == 0:
        ret.append(x)

ret  = [x for x in lst if x % 2 == 0] # 列表解析式 是可以使用if 的

In [55]:
ret

[0, 2, 4, 6, 8]

In [56]:
ret = [x for x in lst if x > 0 and x < 5]

In [57]:
ret

[1, 2, 3, 4]

In [59]:
ret = [x for x in lst if x < 5 or x > 7 if x % 2 == 0 ]

In [60]:
ret

[0, 2, 4, 8]

In [61]:
[x, y for x in range(5) for y in rnge(5, 10)]

SyntaxError: invalid syntax (<ipython-input-61-0065a2cb2413>, line 1)

In [63]:
[(x, y) for x in range(5) for y in range(5, 10)]

[(0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9)]

In [64]:
ret = []
for x in range(5):
    for y in range(5, 10):
        ret.append((x, y))

In [65]:
ret

[(0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9)]

In [ ]:
ret = []
for x in range(5):
    for y in range(5, 10):
        for z in range(10, 15):
            

In [66]:
ret = [(x,y,z) for x in range(5) for y in range(5,10) for z in range(10,15)]


In [67]:
ret

[(0, 5, 10),
 (0, 5, 11),
 (0, 5, 12),
 (0, 5, 13),
 (0, 5, 14),
 (0, 6, 10),
 (0, 6, 11),
 (0, 6, 12),
 (0, 6, 13),
 (0, 6, 14),
 (0, 7, 10),
 (0, 7, 11),
 (0, 7, 12),
 (0, 7, 13),
 (0, 7, 14),
 (0, 8, 10),
 (0, 8, 11),
 (0, 8, 12),
 (0, 8, 13),
 (0, 8, 14),
 (0, 9, 10),
 (0, 9, 11),
 (0, 9, 12),
 (0, 9, 13),
 (0, 9, 14),
 (1, 5, 10),
 (1, 5, 11),
 (1, 5, 12),
 (1, 5, 13),
 (1, 5, 14),
 (1, 6, 10),
 (1, 6, 11),
 (1, 6, 12),
 (1, 6, 13),
 (1, 6, 14),
 (1, 7, 10),
 (1, 7, 11),
 (1, 7, 12),
 (1, 7, 13),
 (1, 7, 14),
 (1, 8, 10),
 (1, 8, 11),
 (1, 8, 12),
 (1, 8, 13),
 (1, 8, 14),
 (1, 9, 10),
 (1, 9, 11),
 (1, 9, 12),
 (1, 9, 13),
 (1, 9, 14),
 (2, 5, 10),
 (2, 5, 11),
 (2, 5, 12),
 (2, 5, 13),
 (2, 5, 14),
 (2, 6, 10),
 (2, 6, 11),
 (2, 6, 12),
 (2, 6, 13),
 (2, 6, 14),
 (2, 7, 10),
 (2, 7, 11),
 (2, 7, 12),
 (2, 7, 13),
 (2, 7, 14),
 (2, 8, 10),
 (2, 8, 11),
 (2, 8, 12),
 (2, 8, 13),
 (2, 8, 14),
 (2, 9, 10),
 (2, 9, 11),
 (2, 9, 12),
 (2, 9, 13),
 (2, 9, 14),
 (3, 5, 10),
 (3, 5, 11),

In [68]:
ret = [x for x in [y for y in range(10)]]

什么时候用列表解析式？

- 跟着感觉走
- 太复杂，需要转换的，就写for 循环
- 多个for循环，明显不知道结果是什么的，就不要用了
- 一眼看不出解析式的结果是什么，也不要用了

In [69]:
# 偶数求平方， 奇数求立方
ret = []
for x in lst:
    if x % 2 == 0:
        ret.append(x ** 2)
    else:
        ret.append(x ** 3)

In [70]:
ret

[0, 1, 4, 27, 16, 125, 36, 343, 64, 729]

In [ ]:
ret = x ** 2 if x % 2 == 0 else x ** 3 # 三元表达式， 三目表达式

In [71]:
# 语法： ret = x if cond else y

In [72]:
# 偶数求平方， 奇数求立方
ret = []
for x in lst:
    ret = 

In [73]:
ret = [x ** 2 if x % 2 == 0 else x ** 3 for x in lst]

In [74]:
ret

[0, 1, 4, 27, 16, 125, 36, 343, 64, 729]

In [75]:
3 if True else 4

3

In [76]:
3 if True # python 中 三元表达式，必须要有else, 且只能有一个if... else

SyntaxError: invalid syntax (<ipython-input-76-b9c192894d7c>, line 1)

In [77]:
3 if True elif False 4

SyntaxError: invalid syntax (<ipython-input-77-a8930d8b886d>, line 1)

### 生成器解析式

In [78]:
range(10000)

range(0, 10000)

In [130]:
[x ** 2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [82]:
g = (x ** 2 for x in range(100)) # 使用一对圆括号 代替之前的 方括号，这就变成了生成器解析式

In [83]:
g

<generator object <genexpr> at 0x1118426d0>

In [107]:
next(g)

529

**生成器解析式，返回的是一个生成器**

优点：不占内存，惰性求值

In [121]:
def fn(x):
    print('executed')
    return x
g = (fn(x) for x in range(10))

In [122]:
g[1]

TypeError: 'generator' object is not subscriptable

In [120]:
next(g)

StopIteration: 

什么时候使用我们的生成器？
- 明确知道需要使用下标的时候，用列表解析式，因为生成器是无法用下标进行访问的
- 如果我们用缓存，就不能用生成器解析式了，就要用列表解析式了


### 集合解析式

In [123]:
{x for x in range(10)}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [126]:
s = {x for x in range(10)} # 返回的是一个set

In [125]:
type(s)

set

### 字典解析式

In [127]:
{str(x): x for x in range(10)} # key 和 value 之间需要用冒号 进行分割，

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [128]:
d = {}
for x in range(10):
    d[str(x)] = x

In [129]:
d

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [132]:
{str(x): x ** 2 for x in range(10)}

SyntaxError: invalid syntax (<ipython-input-132-97fd58ecc4cd>, line 1)

## 可迭代对象和迭代器

In [133]:
r = range(10)

In [134]:
r.__iter__

<method-wrapper '__iter__' of range object at 0x111837c60>

In [135]:
s = '123'

In [136]:
s.__iter__

<method-wrapper '__iter__' of str object at 0x1116876f8>

In [137]:
i = 1

**有\_\_iter\_\_ 方法的对象叫做可迭代对象**

In [139]:
for x in range(10):
    pass

In [140]:
b = b'abc'

In [141]:
b[0]

97

哪些数据结构是可迭代对象：
- list
- set
- tuple
- dict
- str
- bytearray
- bytes

**迭代器是可迭代对象, 可迭代对象不一定是迭代器**

In [145]:
it = iter(range(10)) # 我们可以通过iter() 将可迭代对象转换为 迭代器

In [143]:
it.__next__()

0

有\_\_next\_\_ 方法的 我们成为迭代器

In [144]:
lst = [1, 2]

**next 函数只有迭代器才有， 从迭代器中取出下一个元素**

In [160]:
it = iter(range(5))


In [161]:
next(it)

0

In [162]:
it.__iter__

<method-wrapper '__iter__' of range_iterator object at 0x11189cbd0>

In [163]:
lst = [['m', 1, 2, 3, 4], ['age', 1, 2, 3]]

In [164]:
for x in lst:
    key = x[0]
    for v in x[1:]:
        print(v)

1
2
3
4
1
2
3


In [165]:
for x in lst:
    it = iter(x)
    key = next(it)
    for v in it:
        print(v)

1
2
3
4
1
2
3


for in 循环对于可迭代对象：
- 首先调用iter 方法转化为迭代器
- 然后不断的调用next 方法
- 直到抛出 StopIteration异常

In [ ]:
it = iter(iterable)
while True:
    try:
        next(it)
    except StopIteration:
        return

## 第三周作业

1. 把字符串形式的整数或浮点数转化为int 或者float，**不是int 和 float 函数**

    提示： 可以使用字典
       
2. 移出列表中的重复元素，并保持新列表和原列表的顺序一致

3. 统计文本中各单词出现的次数

4. 把1-4000 之间的任意整数转化为罗马数字